In [1]:
%load_ext autotime

import sys
sys.path.append("../..")

time: 1.19 ms (started: 2021-05-07 21:55:45 -03:00)


In [2]:
import fitparse
import pandas as pd
import plotly.express as px

fit_path = '../../data/pocos.fit'
fitfile = fitparse.FitFile(fit_path)


time: 1.95 s (started: 2021-05-07 21:55:45 -03:00)


In [3]:
import json
with open('../../data/pocos.json', 'r') as fid:
    data = json.load(fid)

records = [el['data'] for el in data if el['type'] == 'record']
df = (pd.DataFrame(records)
        .assign(timestamp=lambda df: pd.to_datetime(df.timestamp))
        .assign(seconds_passed=lambda df: (df.timestamp - df.timestamp[0]).dt.total_seconds())
        )


time: 2.91 s (started: 2021-05-07 21:55:47 -03:00)


In [4]:
fig_df = df.query('seconds_passed < 7000')
px.line(fig_df,
        x='seconds_passed',
        y='speed')

time: 1.41 s (started: 2021-05-07 21:55:50 -03:00)


In [5]:
from scipy.interpolate import interp1d
elevation_interpolator = interp1d(df.distance, df.altitude)


time: 137 ms (started: 2021-05-07 21:55:51 -03:00)


In [9]:
from cadCAD_tools import easy_run
from model import initial_state, params, timestep_block
params.update(elevation_interpolator=[elevation_interpolator],
              dt=[0.5])

results = easy_run(initial_state,
                   params,
                   timestep_block,
                   10000,
                   1,
                   assign_params=False)

Initializing configurations: 100%|██████████| 1/1 [00:00<00:00, 100.82it/s]
                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (10000, 6, 1, 6)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded



ValueError: 'sim_configs' N must > 0

In [10]:
params

{'gravity': [9.81],
 'rolling_resistance': [0.005],
 'wind_resistance': [0.25],
 'mass': [65],
 'dt': [0.5],
 'elevation_interpolator': [<scipy.interpolate.interpolate.interp1d at 0x7f48f29c4c20>]}

time: 2.18 ms (started: 2021-05-07 21:56:11 -03:00)
